In [1]:
from google.colab import drive
drive.mount('/content/drive')

!Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Mounted at /content/drive
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).'


In [ ]:
!pip install transformers
!pip install pytorch_lightning

In [17]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import BertTokenizer
import pytorch_lightning as pl

In [18]:
# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/archive.zip', delimiter='\t',usecols=['comment', 'label_id'],on_bad_lines='skip')

In [19]:
df

,comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1.0
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0.0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1.0
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0.0
4,شیرینی وانیلی فقط یک مدل بود.,0.0
...,...,...
69995,سلام من به فاکتور غذاهایی که سفارش میدم احتیاج...,1.0
69996,سایز پیتزا نسبت به سفارشاتی که قبلا گذشتم کم ش...,1.0
69997,من قارچ اضافه رو اضافه کرده بودم بودم اما اگر ...,0.0
69998,همرو بعد ۲ساعت تاخیر اشتباه آوردن پولشم رفت رو...,1.0


In [20]:
from transformers import AutoTokenizer

# Load the tokenizer for ParsBERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', unk_token="[UNK]")

def preprocess(text, max_length=512):
    # Encode text with ParsBERT tokenizer
    encoded_dict = tokenizer.encode_plus(
                        text,
                        add_special_tokens=True,
                        max_length=max_length,
                        padding='max_length',
                        truncation=True,
                        return_attention_mask=True,
                        return_tensors='pt',
                   )
    
    # Return the input IDs, attention mask, and token type IDs as a dictionary
    return {'input_ids': encoded_dict['input_ids'],
            'attention_mask': encoded_dict['attention_mask'],
            'token_type_ids': encoded_dict['token_type_ids']}
    
    # Return the input IDs, attention mask, and token type IDs as a tuple
    return encoded_dict['input_ids'], encoded_dict['attention_mask'], encoded_dict['token_type_ids']

# preprocess(df["comment"], max_length=512)

df['comment'] = df['comment'].apply(preprocess)

In [10]:

X = df["comment"]
y = df["label_id"]
# Split data into training, validation, and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=42)


class SnappFoodDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        print(self.encodings.keys())  # print out the keys in the encodings dictionary
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SnappFoodDataset(train_texts, train_labels)
test_dataset = SnappFoodDataset(test_texts, test_labels)
model = AutoModelForSequenceClassification.from_pretrained("HooshvareLab/bert-base-parsbert-uncased", num_labels=2)

# Define the training arguments and trainer
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=lambda data: {'input_ids': torch.stack([item['input_ids'] for item in data]),
                                'attention_mask': torch.stack([item['attention_mask'] for item in data]),
                                'labels': torch.stack([item['labels'] for item in data])},
)

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassifica

In [13]:
# Fine-tune the model on the SnappFood dataset
def train():
    trainer = pl.Trainer(min_epochs=1, max_epochs=1, progress_bar_refresh_rate=50)
    try:
        trainer.fit(model, data_module)
    except KeyError as e:
        print(f"KeyError: {e}. The instance causing the error is: {data_module.train_dataset[int(str(e))]}")

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Int64Index([47339, 67456, 12308, 32557,   664, 21003, 29637, 57937, 23680,
            26306,
            ...
            67221, 41090, 16023, 60263, 44131, 37194,  6265, 54886,   860,
            15795],
           dtype='int64', length=56000)


KeyError: ignored